In [3]:
#import numpy as np
#import torch
from matplotlib import pyplot as plt
from jax import grad
import jax.numpy as jnp
import cProfile

RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. You may be able work around this issue by building jaxlib from source.

In [2]:
#!pip install --upgrade pip
!pip install --upgrade "jax[cpu]"

     |████████████████████████████████| 745 kB 286 kB/s eta 0:00:01
  Using cached absl_py-0.13.0-py3-none-any.whl (132 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 45.0 MB 310 kB/s eta 0:00:01
  Created wheel for jax: filename=jax-0.2.20-py3-none-any.whl size=854826 sha256=1671472e50862a0a2fcc1e40f6a17b5812bdfa18a644ba96ef646021f4bb0e4f
  Stored in directory: /Users/brandonbrown/Library/Caches/pip/wheels/2a/42/43/b115ab29c5326e269bfafd5e4abb5d94f04670ab8b865d4f1b
Successfully built jax


In [2]:
# Generates a random project matrix
def gen_rand_proj(d,size=1):
    opts = [-1,1]
    #neg_one_p = 1/(2*np.sqrt(d))
    #zero_p = (1-(1/np.sqrt(d)))
    probs = [0.5,0.5]
    w_ = np.random.choice(opts,size=size,p=probs)
    #TODO: make this a sparse matrix
    return w_

In [3]:
gen_rand_proj(5,(10,10))

array([[ 1,  1,  1,  1,  1,  1, -1,  1,  1, -1],
       [ 1,  1,  1, -1, -1,  1,  1, -1, -1, -1],
       [-1,  1, -1, -1,  1, -1, -1, -1,  1,  1],
       [ 1,  1, -1, -1, -1,  1,  1, -1,  1, -1],
       [-1,  1, -1, -1, -1, -1, -1,  1,  1, -1],
       [ 1, -1,  1,  1, -1, -1,  1,  1, -1,  1],
       [ 1,  1, -1,  1,  1,  1,  1, -1,  1,  1],
       [ 1,  1,  1, -1, -1,  1, -1,  1,  1,  1],
       [ 1,  1,  1,  1, -1, -1,  1, -1, -1,  1],
       [ 1, -1,  1, -1,  1,  1, -1, -1,  1,  1]])

In [115]:
def test1(x,W):
    return torch.sum(x @ W)

def test2(x,W,S):
    return torch.sum(x @ W[:,S])

In [107]:
dim1,dim2 = 784,20000
x = torch.randn(1,dim1);
W1 = torch.randn(dim1,dim2,requires_grad=True)
W2 = torch.randn(dim1,dim2,requires_grad=True);
S = torch.randint(0,dim2,(50,));

In [73]:
cProfile.run("test1(x,W1)") #0.003 seconds -> 0.009 -> 0.016

         5 function calls in 0.012 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.012    0.012    0.012    0.012 <ipython-input-38-88928335a61d>:1(test1)
        1    0.000    0.000    0.012    0.012 <string>:1(<module>)
        1    0.000    0.000    0.012    0.012 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [76]:
cProfile.run("test2(x,W2,S)") #0.006 seconds -> 0.005 -> 0.006

         5 function calls in 0.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005    0.005    0.005 <ipython-input-38-88928335a61d>:4(test2)
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        1    0.000    0.000    0.005    0.005 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [110]:
cProfile.run("test1(x,W1).backward()") # 0.016 seconds -> 0.078 -> 0.144

         15 function calls in 0.015 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.007    0.007    0.007    0.007 <ipython-input-38-88928335a61d>:1(test1)
        1    0.000    0.000    0.015    0.015 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:20(_make_grads)
        1    0.000    0.000    0.007    0.007 __init__.py:44(backward)
        1    0.000    0.000    0.007    0.007 tensor.py:167(backward)
        1    0.000    0.000    0.015    0.015 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {built-in method ones_like}
        1    0.000    0.000    0.000    0.000 {built-in method sum}
        1    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000   

In [109]:
cProfile.run("test2(x,W2,S).backward()") #0.023 seconds -> 0.065 -> 0.220

         15 function calls in 0.060 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005    0.005    0.005 <ipython-input-38-88928335a61d>:4(test2)
        1    0.000    0.000    0.059    0.059 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:20(_make_grads)
        1    0.000    0.000    0.054    0.054 __init__.py:44(backward)
        1    0.000    0.000    0.054    0.054 tensor.py:167(backward)
        1    0.000    0.000    0.060    0.060 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {built-in method ones_like}
        1    0.000    0.000    0.000    0.000 {built-in method sum}
        1    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000   